# Assignment 10: Collider with RF cavity

<html>
<div class="alert alert-info" role="alert" style="margin-top: 10px">
    <ul>
        <li>Parse MADX file of a collider</li>
        <li>Implement thin RF cavity trasfer matrix and insert into collider</li>
        <li>Track Twiss and dispersion</li>
    </ul>
</div>
</html>

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>If you use google colab, run this cell:</strong>
    </div>
</html>

In [ ]:
# for google colab, run this cell
!git clone https://github.com/potato18z/pam1-hs2021.git
import sys
sys.path.append('./pam1-hs2021')

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>If you run it locally, run</strong>
               </div>
</html>

```bash
$ cd .../pam1-hs2021
...pam1-hs2021$ git pull
```
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
to get the updated repository.</div></html>

## Load MADX file
A PSI colleague prepared a fantasy collider example in MAD-X. In the first cell we parse this file using our ```MADXParser``` class. This class provides some useful functions and attributes for you:
```Python
madx = MADXParser()
    
# fn is the MAD-X file to parse
madx.parse(fn)
    
# returns all elements in ascending order in a list.
madx.getBeamline()

# returns an instance of the particle, e.g. proton, electron, etc.
madx.getParticle()

# returns the total energy [GeV]
madx.getEtot()
```

In [ ]:
from AcceLEGOrator.MADXParser import *

try:
    madx = MADXParser()
    
    madx.parse('madx/collider_madx.mad')
    
    print(madx)
    
    beamline = madx.getBeamline()
    
except Exception as e:
    print(e)

## Thin RF-Cavity
From the Eq. (36) of `pam-s9` we know
$$
R =
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & -\frac{1}{f_{||}} & 1 \\
\end{pmatrix},
$$
where
$$
\frac{1}{f_{||}} = \frac{q\hat{V}}{P_0c}k\cos\phi_0,
$$
with voltage $\hat{V}$, $k=2\pi f_{RF}/c$ where $f_{RF}$ is the RF frequency, and $\phi_0$ is the RF phase.

Note that we need to calculate the energy gain through the cavity, and change the gamma factor accordingly. The energy gain due to the RF-Cavity is given by
$$
\Delta W = |q|\hat{V}T\cos(\phi_0),
$$
with voltage $\hat{V}$, charge $q$, RF phase $\phi_0$ and transit time factor
$$
T = \frac{2\beta_0}{\pi}\sin\left(\frac{\pi}{2\beta_0}\right).
$$

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO</strong>
        <ol>
            <li>Implement the Thin RF-Cavity map.</li>
        </ol>
               </div>
</html>

In [ ]:
from AcceLEGOrator import Map, Physics, Constants
from AcceLEGOrator import Parameter as param
import numpy as np

class ThinRfCavity(Map):
    
    # param voltage [V]
    # param freq RF frequency [MHz]
    # param phase [rad]
    def __init__(self, V, freq, phase):
        
        self.V = V
        self.freq = freq
        self.phase = phase
        
        beta_0  = Physics.getBeta(param.gamma_0)
        
        c = Constants.clight
        
        #  [k] = 1 / m
        k = ...
        
        # [P0] = eV / c
        P0 = param.mass * 1e6 * param.gamma_0 * beta_0
        
        # [f_inv] = m
        f_inv = ...
        
        R = np.matrix([[1, 0, 0, 0, 0, 0],
                       [0, 1, 0, 0, 0, 0],
                       [0, 0, 1, 0, 0, 0],
                       [0, 0, 0, 1, 0, 0],
                       [0, 0, 0, 0, 1, 0],
                       [0, 0, 0, 0, ..., 1]])
        
        # transit time factor
        T = ...
        
        # energy gain [MeV]
        W = ...
        
        # new relativistic factor and logging
        self.gamma_before = param.gamma_0
        param.gamma_0 += W / param.mass
        self.gamma_after = param.gamma_0
        
        super(ThinRfCavity, self).__init__(R, 0)
    
    
    def __str__(self):
        return 'ThinRfCavity(V = ' + str(self.V) + ' [V], '\
               + 'rf freq = ' + str(self.freq) + ' [MHz], '\
               + 'rf phase = ' + str(self.phase) + ' [rad])\n'
    
    def get(self, length):
        return ThinRfCavity(self.V, self.freq, self.phase)

<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        Our collider will have two RF-Cavities with $\hat{V} = 0.001$ MV, $\phi = 0$ and
        $f_{rf} = 10^{-4}$ MHz.
    </div>
</html>

In [ ]:
RF = ThinRfCavity(V=1000, freq=1e-4, phase=0)

## Find where to insert the cavities by tracking the dispersion

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ol>
            <li>Use the function `trackDispersion` in `Tracking.py` to track the dispersion with the following initial values. Since we alreadyd have the entire beamline, we should set cell=beamline and n_cells=1.</li>
            $$\begin{pmatrix}D_x \\ D_{p_x} \\ 1 \end{pmatrix}
            =\begin{pmatrix} 0.5069938765 \\ −0.1681363086 \\ 1\end{pmatrix}$$
            <li>plot the result</li>
        </ol>
    </div>
</html>

You can find more details about dispersion in
[this link](https://indico.cern.ch/event/471931/contributions/1149878/attachments/1214721/1794333/Lectures.pdf)

(For a FODO cell, the calculation of initial dispersion is given in Slide 119)

In [ ]:
from AcceLEGOrator import Tracking
initial_dispersion = [0.5069938765, -0.1681363086, 1]
all_dispersions, lengths = Tracking.trackDispersion(...)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(dpi=300)
fig.set_size_inches(9,3)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel('Dispersion [m]')

axis.plot(lengths, ..., label=r'$D_x$', color='lightgreen')
axis.plot(lengths, ..., label=r'$D_{p_x}$', color='gray')
axis.legend()

plt.show()

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ol>
            <li>Locate the middle index of the two regions in the beamline where dispersion is zero</li>
            <li>Insert the above defined Thin RF-Cavity to the two locations.</li>
        </ol>
    </div>
</html>

In [ ]:
# find where all_dispersions[:,0] are close to zero, then find the middle point
index1 = ...
index2 = ...
beamline.insert(index1, RF)
beamline.insert(index1, RF)

## Tracking of $\beta$

We will repreduce this plot:
![twiss_dispersion_rf](img/twiss_dispersion_rf.png)

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Use `trackTwiss` and `trackDispersion` in `Tracking.py` to reproduce the above plot. Now our beamline has been changed, so we need to redo the tracking.</li>
        </ul>
    </div>
</html>

In [ ]:
all_twiss, lengths = Tracking.trackTwiss(..., twiss_init=None)
all_dispersions, _ = Tracking.trackDispersion(...)

fig = plt.figure(dpi=150)
fig.set_size_inches(6,6)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel(r'$\beta_x(m), \beta_y(m), D_x(m)$')

axis.plot(lengths, ..., label=r'$\beta_x$', color='black')
axis.plot(lengths, ..., label=r'$\beta_y$', color='red')
axis.plot(lengths, ..., label=r'$D_x$', color='lightgreen')
axis.set_xlim([0, 250])
axis.set_ylim([-50, 450])
axis.legend()

plt.show()

## Energy change

Note that we have define `gamma_before` and `gamma_after` for the RF cavity. This is to log the energy change due to the cavity.
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>For the two cativies in our beamline, print the gamma factors before and after them to see the energy change.</li>
            <li>*Bonus: Plot the energy against lengths s. You should get a 2-step ladder shape.</li>
        </ul>
    </div>
</html>

In [ ]:
RF_1 = beamline[index1]
RF_2 = beamline[index2]
#print(...)